In [1]:
%pip install selenium


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/48/5d/acf5905c36149bbaec41ccf7f2b68814647347b72075ac0b1fe3022fdc73/tqdm-4.66.5-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.4 kB ? eta -:--:--
   ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
import csv
from tqdm import tqdm
import os
import pandas as pd

In [2]:
# def category_csvfile(title, link):
#     driver = webdriver.Chrome()

#     driver.get(link)
#     time.sleep(1)

#     list_container = driver.find_element(By.CLASS_NAME, 'List_List_container__JnQMS')

#     while True:
#         prev_li_count = len(list_container.find_elements(By.TAG_NAME, 'li'))
        
#         html = driver.find_element(By.TAG_NAME, 'html')
#         html.send_keys(Keys.END)
#         time.sleep(2)



#         current_li_count = len(list_container.find_elements(By.TAG_NAME, 'li'))

#         if current_li_count == prev_li_count:
#             time.sleep(10)
#             reset_count = len(list_container.find_elements(By.TAG_NAME, 'li'))
#             print("break " + str(reset_count))
#             if reset_count == prev_li_count:
#                 break

#     list_items = list_container.find_elements(By.TAG_NAME, 'li')

#     print("total amount is " + str(len(list_items)))
    
#     data = []
#     if os.path.isfile('job_link.csv'):
#         with open('job_link.csv', 'r', newline='') as file:
#             reader = csv.reader(file)
#             data = list(reader)
        

#     with open('job_link.csv', mode='w', newline='') as file:
#         writer = csv.writer(file)
#         for item in list_items:
#             card = item.find_element(By.CLASS_NAME, 'Card_className__u5rsb')
#             anchor = card.find_element(By.TAG_NAME, 'a')
#             link = anchor.get_attribute('href')
#             data[1:] += [(title, link)]
#             # writer.writerow([title, link])
#         writer.writerows(data)
    

#     # driver.quit()

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def collect_by_page(page):

    driver = webdriver.Chrome()
    driver.get(f'https://www.gov.kr/portal/faq?&pageIndex={page}')

    # tbody 안의 모든 tr 요소 가져오기
    fqa_table = driver.find_element(By.CLASS_NAME, 'md')
    fqa_table_body = fqa_table.find_element(By.TAG_NAME, 'tbody')
    fqa_table_list = fqa_table_body.find_elements(By.TAG_NAME, 'tr')

    fqa_list = []

    # 각 fqa 들어가서 데이터 가져오기 
    for row in fqa_table_list:
        link_element = row.find_element(By.XPATH, './/td[@class="subj m-show"]/a')
        # tr 안의 a 태그에서 href 속성을 가져오기
        href = link_element.get_attribute('href')
        driver.get(href)
        time.sleep(4) 

        try:

            # 페이지에서 필요한 데이터를 추출
            fqa_title = driver.find_element(By.TAG_NAME, 'h2').text 
            fqa_category = driver.find_element(By.CLASS_NAME, 'view-title').find_element(By.TAG_NAME, 'strong').text
            fqa_contents_div = driver.find_element(By.CLASS_NAME, 'view-contents')
            fqa_contents_text = fqa_contents_div.text # 해당 div 내의 텍스트 추출

            # 혹시나 걸려있는 link가 있으면 그 링크를 제공해주기 위한 코드
            links = []
            for element in fqa_contents_div.find_elements(By.XPATH, ".//*"):
                if element.tag_name == "a":  # 요소가 링크(<a>)라면
                    link = element.get_attribute('href')
                    if href:  # href가 존재하면
                        links.append(link)



            print(fqa_title)

            fqa_list.append({'category':fqa_category,
                            'question':fqa_title, 
                            'answer':fqa_contents_text,
                            'links':links})
        except Exception as e:
            print(f"데이터가 없습니다.")

        # 원래 페이지로 돌아가기
        driver.back()
        time.sleep(4) 

    return fqa_list



In [6]:
total_frq_list = []
for i in range(38):
    print(i)
    total_frq_list = total_frq_list + collect_by_page(i+1)

df = pd.DataFrame(total_frq_list)
df.to_csv('gov_faq_data.csv', index=True, encoding="utf-8-sig")

0
브라우저 캐시 및 쿠키 삭제 방법을 알려주세요.
해외에서 해외체류신고시 PC 시간대 설정 방법을 알려주세요.
정부24 연말정산 페이지 접속 시 ip 차단되었다고 나올때 조치방법
[취학통지서] 취학통지서 온라인 발급은 꼭 세대주만 가능한가요?
[전입신고] 전입지 세대주 확인까지 완료되었는데 처리상태가 미접수로 확인됩니다.
Anysign4PC 프로그램 반복설치 오류가 발생하는 경우 조치 방법
[전입신고] 전입신고 시 신청한 요금감면 일괄신청 내역 확인은 어떻게 하나요?
[취학통지서] 공동학구인 경우 취학가능 학교가 여러개 인데 어느 학교에 입학해야 하나요?
[취학통지서] 취학통지서 통학구역에 단일학구와 공동학구 표시가 없습니다.
[취학통지서] 취학통지서에 학교 예비소집 날짜가 여러 개 기재되어 있습니다. 각 날짜별로 어느 학교의 예비소집 날짜인지 어떻게 알 수 있나요?
1
[취학통지서] 발급기간 경과 이후 취학통지서 분실(온라인발급, 등기우편발급) 시 재발급은 가능한가요?
[취학통지서] 취학통지서를 온라인으로 신청은 하였으나, 신청기간 내에 발급(인쇄)하지 않은 경우 어떻게 해야 하나요?
[취학통지서] 온라인으로 발급받았는데 왜 또 등기우편을 보내주나요?
[취학통지서] 취학통지서를 온라인으로 발급받으면 등기우편은 안 오나요?
[취학통지서] 취학통지서 대상자나 '조회된 사항이 없습니다. 취학아동 보호자 및 대상자를 확인하세요'라고 뜹니다. 어떻게 해야 하나요?
[취학통지서] 정부24에서 취학통지서 발급 신청 중 세대주/보호자가 아니라고 나옵니다. 어떻게 해야 하나요?
[취학통지서] 정부24에서 취학통지서를 발급(출력)했는데, 훼손된 경우 어떻게 해야 하나요?
[취학통지서] 온라인출력물 발행번호가 표기되어 있지 않아요?
[지방직 공무원] 취업서류 일괄제출 서비스 활용 안내
여권재발급 시 사진관련 오류 사항 ( 에러코드 : 701 ) 사진품질 검사
2
여권재발급 시 사진관련 오류 사항 ( 에러코드 : 707 ) 여권용 사진 규격 준수여부
여권재발급 시 사진관련 오류

In [7]:
import pandas as pd
file_path = 'gov_faq_data.csv'
df = pd.read_csv(file_path, index_col=0)

print(df)

    category                                           question  \
0       환경설정                         브라우저 캐시 및 쿠키 삭제 방법을 알려주세요.   
1       민원신청                  해외에서 해외체류신고시 PC 시간대 설정 방법을 알려주세요.   
2       환경설정              정부24 연말정산 페이지 접속 시 ip 차단되었다고 나올때 조치방법   
3       민원발급                [취학통지서] 취학통지서 온라인 발급은 꼭 세대주만 가능한가요?   
4       전입신고       [전입신고] 전입지 세대주 확인까지 완료되었는데 처리상태가 미접수로 확인됩니다.   
..       ...                                                ...   
371     전자결제                                        부가수수료가 뭔가요?   
372     전자결제                                    수수료는 어떻게 구성되나요?   
373     전자결제                                 결제방법에는 어떤 종류가 있나요?   
374     민원발급  ‘현재 보안설정으로는 이 페이지의 ActiveX 컨트롤을 실행할 수 없습니다.’라는...   
375     민원발급  스캔은 되나 발급문서 진위확인(스캐너용) 프로그램에서 스캔영역, 원본영역에 아무것도...   

                                                answer links  
0    캐시 및 쿠키 삭제\n\n○ Edge의 경우\n1. 현재 사용중인 모든 브라우저 창...    []  
1    윈도우 시작 > 설정 > 시간&언어 > 날짜 & 시간으로 이동합니다.\n\n표준 시...    []  
2    해

In [16]:
category_list = []

for category in df['category'].unique():
    category_list += category.split(" | ")

print(list(set(category_list)))

['전자결제', '인증', '주민등록', '스마트폰', '기타민원', '토지대장', '정부서비스', '건축물대장', '회원관리', '환경설정', '보조금24', '전입신고', '생활정보', '처리결과', '민원발급', '민원신청']


['전자결제', '인증', '주민등록', '스마트폰', '기타민원', '토지대장', '정부서비스', '건축물대장', '회원관리', '환경설정', '보조금24', '전입신고', '생활정보', '처리결과', '민원발급', '민원신청']

In [ ]:
['전입신고', '주민등록', '민원발급', '회원관리', '처리결과', '민원신청', '보조금24', '인증', '전자결제', '환경설정', ]